In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

from Unit_2.dataFilePath import *

# salary data
salaryData = pd.read_csv(salaryPath)
salaryData.set_index(salaryData["State"], inplace = True)
salaryData.drop(columns=["State"], inplace = True)

salaryData.plot.line()

plt.show()




ModuleNotFoundError: No module named 'Unit_2'